In [46]:
import pandas as pd
from tqdm import tqdm
from collections import Counter

In [47]:
# TODO 추천에 활용하지 않을 category 목록들을 필터링합니다.
# TODO 필터링 이후, similarity에 대한 BFS 알고리즘을 적용합니다 -> 이전에 구한 similarity가 전체 item을 기준으로 구해졌기 때문에, 예외 처리를 해줘야 합니다.
# TODO BFS 알고리즘을 통해 찾아낸 clustering group을 만들어주고, "ETC"나 "etc"로 group된 item들은 -1로 변환합니다.
# TODO house_interaction에 클러스터 결과를 merge해줍니다. 이때, 클러스터 그룹이 -1이거나 house_interaction에만 포함된 item들은 cluster_id.max() + 1을 더해줍니다.
# TODO model을 학습해서 학습한 결과를 backend에 반영합니다.
# TODO cluster-major-item list를 생성합니다.

In [48]:
item = pd.read_csv("similarity_item_99243.csv")

In [49]:
item.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list
0,226754,유아·아동|완구·교구|신생아·영아완구,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,0 개월 한글 영어,FINDEN BEBE(핀데베베),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가799,000사이즈정보없음출처smartstore.naver.com",0~36개월/한글영어,[]


In [50]:
del_list = [
"공구",
"욕실용품",
"유아·아동",
"청소용품",
"캠핑·레저",
"반려동물",
"생필품",
"렌탈",
"생활용품",
"식품",
"인테리어시공",
"칼",
"선풍기",
"컵·잔·텀블러",
"조리도구·도마",
"홈갤러리",
"전기요·온수매트",
# "에어컨",
"커피·티용품",
"조명부속품",
"센서등",
"복합기·프린터·스캐너",
"소파·생활커버",
"이미용가전",
"주방잡화",
"그릇·홈세트",
"후크·수납걸이",
"데스크·디자인문구",
"냄비·프라이팬·솥",
"베이킹용품",
# "세탁기·건조기",
"거울",
"매트리스·토퍼",
"파티·이벤트용품",
"해충퇴치기",
"다이어리·플래너",
"수세미걸이·세제통",
"다리미·보풀제거기",
"멀티탭·공유기정리함",
"멀티탭·공유기정리함",
"다리미·보풀제거기",
"디자인문구",
"보드게임",
"리빙박스·바구니",
# "가습기",
"전기히터·온풍기",
"전기·멀티포트"
]

In [51]:
item.category.fillna("|", inplace=True)

In [52]:
import pickle
with open("similarity_99243.pickle", "rb") as pkl:
    data = pickle.load(pkl)

item["similarity_list"] = data

In [53]:
item_ = item[(~item.category.str.contains("|".join(del_list))) & ~(item.category=="|")]

In [54]:
len(item_)

88634

In [55]:
(item_.iloc[6].similarity_list)

[['002 애쉬 인 소파 페 브릭', '002 애쉬 인 소파 페 브릭', 69359, 1.0]]

In [56]:
item_.sort_values("item", inplace=True)

/tmp/ipykernel_44871/1535190289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_.sort_values("item", inplace=True)


In [57]:
list(item_[item_.item == 356808].similarity_list)

[[['쿠키앤크림 사다리 책장 단', '쿠키앤크림 사다리 책장 종 택', 25490, 0.8666666666666667]]]

In [58]:
CLUSTER_ID = 0
NON_CLUSTER_ID = -1

lim = 5
threshold = 0.8

# TODO threshold 이상의 similarity 가지는 아이템을 저장하는 list 칼럼 생성
def get_item(x, threshold):
    tmp = []
    for a in x:
        if a[3] >= threshold:
            tmp.append(a[2])
    return tmp

item_["similar_item_list"] = item_.similarity_list.apply(lambda x:get_item(x, threshold))

/tmp/ipykernel_44871/3209520921.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["similar_item_list"] = item_.similarity_list.apply(lambda x:get_item(x, threshold))


In [59]:
item_[item_.item == 356808]

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list
126402,356808,가구|진열장·책장|책장,별점 4.6점,(685)개 리뷰,"91,100원",쿠키앤크림 사다리 책장 단,채우리,56%,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,NaN,쿠키앤크림 사다리책장 4단 2colors,"[[쿠키앤크림 사다리 책장 단, 쿠키앤크림 사다리 책장 종 택, 25490, 0.8...",[25490]


In [60]:
graph = {item_id:sim_items for item_id, sim_items in zip(item_.item, item_.similar_item_list)}
visited = {item_id:NON_CLUSTER_ID for item_id in item_.item}

In [61]:
# 양방향 그래프로 만들기
for item_id, sim_items in graph.items():
    for item_id_ in sim_items:
        try:
            graph[item_id_].append(item_id)
        except:
            pass

In [62]:
len(graph)

88634

In [63]:
len(visited)

88634

In [64]:
cnt = 0
for g in graph.values():
    if len(g) > 1:
        cnt += 1
cnt

14999

In [65]:
def BFS(item_id:int, cls_id):
    from collections import deque
    q = deque([(item_id, 0)])
    visited[item_id] = cls_id
    while(q):
        node, hop = q.popleft()
        if hop > lim:
            return
        try:
            for id in graph[node]:
                if visited[id] == -1: # 아직 클러스터링되지 않은 아이템인 경우
                    visited[id] = cls_id
                    q.append((id, hop + 1))
        except:
            pass

In [66]:
for item_id in tqdm(graph.keys()):
    if visited[item_id] == NON_CLUSTER_ID and len(graph[item_id]) > 0:
        BFS(item_id, CLUSTER_ID)
        CLUSTER_ID += 1

100%|██████████| 88634/88634 [00:00<00:00, 538559.52it/s]


In [67]:
item_["cls_id"] = item_.item.map(visited)

/tmp/ipykernel_44871/1113527780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["cls_id"] = item_.item.map(visited)


In [68]:
item_.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
2215,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,[],[],-1


In [69]:
item_.loc[item_.preprocessed_title.str.contains("etc|ETC"), "cls_id"] = -1

In [70]:
cnt  = 0
for k, v in Counter(visited.values()).items():
    if v == 1:
        cnt += 1
cnt

493

In [71]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

493

In [72]:
for item_id in not_clustered_item:
    # 자신과 이웃인 item의 cluster_group을 찾아서 바꿔주기
    group_candidates = []
    for neighbor_item_id in graph[item_id]:
        try:
            group_candidates.append(visited[neighbor_item_id])
        except:
            pass
    CNT = Counter(group_candidates)
    if len(CNT) >= 2:
        tmp = CNT.most_common(2)
        if tmp[0][0] == -1: # 가장 빈도 높은 클러스터가 클러스터링이 안되어있을 때 : -1
            final_cls_id = tmp[1][0] # 나머지 모든 item을 2등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
        else:
            final_cls_id = tmp[0][0] # 나머지 모든 item을 1등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
    elif len(CNT) == 1:
        tmp = CNT.most_common(1)
        if tmp[0][0] == -1: # 클러스터가 클러스터링이 안되어있을 때 : -1
            final_cls_id = tmp[1][0] # 나머지 모든 item을 not_clustered_item의 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
        else:
            final_cls_id = tmp[0][0] # 나머지 모든 item을 1등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
    else: # 이웃들이 모두 잘려나간 카테고리에 포함되는 경우 -> -1로 처리
        visited[item_id] = -1

In [73]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

2

In [74]:
for item_id in not_clustered_item:
    visited[item_id] = -1

In [75]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

0

In [76]:
# 모든 클러스터를 처리함

In [77]:
len(visited)

88760

In [78]:
len(item_)

88634

In [79]:
# 개수 차이가 나는 이유는 visited의 일부가 잘려나간 카테고리에 포함되기 때문
item_["cls_id"] = item_.item.map(visited)

/tmp/ipykernel_44871/1011874532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["cls_id"] = item_.item.map(visited)


In [80]:
len(item_[item_.cls_id != -1])
# 전체 88634 item 중에 25039개에 해당하는 아이템을 클러스터링함. (28.2%)

25039

In [81]:
item_.sample(10)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
20731,10713,가구|거실장·TV장|일반거실장,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,media console white,엘름,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가215,000사이즈정보없음출처www.ellm.co.kr",Media Console - White,[],[],-1
64737,151236,데코·식물|플라워·식물|화병·화분|화분,별점 4.3점,(4)개 리뷰,NaN,모던 시멘트 테라 화분 도트벨,먼데이팟,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가7,700사이즈정보없음출처정보없음",모던 시멘트 테라조 화분 도트벨_4sizes,[],[],-1
139573,639818,데코·식물|시계|벽시계,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,플라워 아크릴 벽 시계,썬데이감성,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,NaN,플라워 아크릴 벽시계,[],[],-1
70536,769500,조명|천장등|펜던트조명,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,밀키웨이 팬던트,그린라이트,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가433,000사이즈정보없음출처led76.com",밀키웨이 5등/6등 팬던트,"[[밀키웨이 팬던트, 밀키웨이 팬던트, 769503, 1.0]]","[769503, 769503]",7361
104092,294306,가구|테이블·식탁·책상|거실·소파테이블,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,오피스 단 소파 티 테이블 cot60010roa cot60070roa,리바트,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가125,000사이즈정보없음출처smartstore.naver.com",오피스 600G 2단 소파 티테이블 COT60010ROA / COT60070ROA...,[],[],-1
145201,511457,수납·정리|서랍장·트롤리|트롤리·이동식선반,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,호텔 트롤리 솔리드 우드 커피 테이블 목적 선반 디스플레이 랙 가정 용 더블 레이어,알리익스프레스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈정보없음출처정보없음,호텔 트롤리 솔리드 우드 커피 테이블 다목적 선반 디스플레이 랙 가정용 더블 레이어,[],[],-1
135099,51913,패브릭|러그·카페트|페르시안러그,별점 4.7점,(49)개 리뷰,"62,000원",페르시안 사프란 카페트,바이빔,38%,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,NaN,페르시안 사프란 카페트 4size,"[[페르시안 사프란 카페트, 페르시안 샤반 카페트, 51910, 0.78260869...","[36009, 84887, 84887]",1058
98165,1696084,패브릭|블라인드·롤스크린|블라인드,별점 5.0점,(4)개 리뷰,NaN,알루미늄 블라인드,아라크네,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,NaN,알루미늄블라인드 34colors,"[[알루미늄 블라인드, 알루미늄 블라인드, 819244, 1.0], [알루미늄 블라...","[819244, 877607, 111361, 13247, 25838, 138310]",598
120409,1547377,가구|의자|인테리어의자,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,체어,코헨,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가176,000사이즈정보없음출처www.thecohen.co.kr",체어29,"[[체어, 체어, 83581, 1.0], [체어, 체어, 201902, 1.0], ...","[83581, 201902, 69210, 51261, 279122, 346391, ...",47
42764,98984,패브릭|러그·카페트|사이잘룩러그,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,네이처 260 놀이 방 매트,크림하우스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가226,000사이즈정보없음출처www.creamhaus.com",네이처 260 놀이방매트,[],[],-1


In [82]:
type(item_[item_.cls_id!=-1].iloc[0]["similar_item_list"])

list

In [83]:
import numpy as np

item_["similar_item_list"] = item_["similar_item_list"].apply(lambda x: np.NaN if len(x) == 0 else "|".join(map(str,x)))

/tmp/ipykernel_44871/3677011072.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["similar_item_list"] = item_["similar_item_list"].apply(lambda x: np.NaN if len(x) == 0 else "|".join(map(str,x)))


In [84]:
item_[item_.cls_id!=-1]

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
18053,55,가구|선반|스탠드선반,별점 4.7점,(409)개 리뷰,NaN,lerberg 철제 선반 l,이케아,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,사이즈60.0 × 35.0 × 148.0 (cm)출처정보없음,LERBERG 철제선반 L 4colors,"[[lerberg 철제 선반 l, lerberg 철제 선반 s, 769, 0.933...",769|236215|769|7390|133699|133700|153789|15379...,0
84812,89,주방용품|주방수납·정리|주방수납용품,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,세라믹 단 수저 통,리벤스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가8,900사이즈27.0 × 8.0 × 12.0 (cm)출처www.11st.co.kr",[품절]세라믹 4단 수저통,"[[세라믹 단 수저 통, 세라믹 물 단 수저 통, 1017200, 0.9090909...",1017200|743437|743437|1017200,1
91253,103,패브릭|매트|주방·다용도매트,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,스트라이프 다용 러그,모던하우스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가29,900사이즈정보없음출처modernhouse.elandmall.com",[단종]스트라이프 다용도 러그,"[[스트라이프 다용 러그, 스트라이프 베드 러너, 380, 0.72727272727...",350421|350421,2
132424,107,주방용품|주방수납·정리|주방수납용품,별점 5.0점,(11)개 리뷰,NaN,트리플 스퀘어 수저 통,모던하우스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가17,900사이즈정보없음출처modernhouse.elandmall.com",트리플 스퀘어 수저통,"[[트리플 스퀘어 수저 통, 트리플 스퀘어 수저 통 레드, 1436, 0.88888...",1436|1436,3
47540,108,주방용품|주방수납·정리|주방수납용품,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,데 일리 단 수저,모던하우스,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가7,900사이즈18.0 × 9.0 × 13.0 (cm)출처modernhouse...",데일리 2단 수저꽂이,[],143,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97177,1808150,수납·정리|화장대·테이블정리|화장품정리용품,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,아크릴 화장 정리 함,한샘,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈정보없음출처mall.hanssem.com,아크릴 화장대 정리함,"[[아크릴 화장 정리 함, 아크릴 화장품 정리 함 소형, 150602, 0.8461...",150602|337023|337060|1488694,2829
125735,1811298,패브릭|쿠션·방석|쿠션,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,코튼 캔버스 쿠션 커버,h&m home,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가7,900사이즈정보없음출처www2.hm.com",코튼 캔버스 쿠션 커버,"[[코튼 캔버스 쿠션 커버, 코튼 캔버스 쿠션 커버, 249323, 1.0], [코...",249323|1224778|537571|1677025|28642|441643|516...,821
22242,1814392,가전·디지털|생활·건강가전|기타생활·건강가전,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,nest learning thermostat,Google,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈정보없음출처store.google.com,[해외]Nest Learning Thermostat,"[[nest learning thermostat, nest thermostat e,...",1609518,9156
32440,1817818,가구|테이블·식탁·책상|식탁|식탁·입식테이블,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,volini ceramic table,플랫포인트,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈정보없음출처www.flatpoint.co.kr,VOLINI CERAMIC TABLE,"[[volini ceramic table, volini ceramic table 1...",1203315|1132073,8611


In [85]:
item_.drop(columns=["similarity_list"], inplace=True)

/tmp/ipykernel_44871/4098756355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_.drop(columns=["similarity_list"], inplace=True)


In [86]:
item_.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similar_item_list,cls_id
2215,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,NaN,-1


In [87]:
from time import time

now = str(round(time()))[5:]
item_.to_csv(f"clustered_item_{now}.csv", index=False)